## ​step 1: Loading dataset and preprocessing it

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/Data/quotes.csv')
data

,Unnamed: 0,quote,author,tags
0,0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"['be-yourself', 'gilbert-perreira', 'honesty',..."
1,1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"['best', 'life', 'love', 'mistakes', 'out-of-c..."
2,2,“Two things are infinite: the universe and hum...,Albert Einstein,"['human-nature', 'humor', 'infinity', 'philoso..."
3,3,"“So many books, so little time.”",Frank Zappa,"['books', 'humor']"
4,4,“A room without books is like a body without a...,Marcus Tullius Cicero,"['books', 'simile', 'soul']"
...,...,...,...,...
2503,2995,“Morality is simply the attitude we adopt towa...,"Oscar Wilde,","['morality', 'philosophy']"
2504,2996,“Don't aim at success. The more you aim at it ...,"Viktor E. Frankl,","['happiness', 'success']"
2505,2997,"“In life, finding a voice is speaking and livi...",John Grisham,['inspirational-life']
2506,2998,"“Winter is the time for comfort, for good food...",Edith Sitwell,"['comfort', 'home', 'winter']"


In [ ]:
# data[data['quote']=='Life is the flower of which love is the honey. The more you give, the more you get.']
data['tags'].unique()

array(["['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
       "['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
       "['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
       ..., "['morality', 'philosophy']", "['comfort', 'home', 'winter']",
       "['just-listen', 'loud', 'owen', 'sara-dessen', 'silence']"],
      dtype=object)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/Data/quotes.csv')
data['quote'] = data['quote'].str.lower()
data['tags'] = data['tags'].str.lower()
data = data.drop('Unnamed: 0',axis=1)
data

,quote,author,tags
0,“be yourself; everyone else is already taken.”,Oscar Wilde,"['be-yourself', 'gilbert-perreira', 'honesty',..."
1,"“i'm selfish, impatient and a little insecure....",Marilyn Monroe,"['best', 'life', 'love', 'mistakes', 'out-of-c..."
2,“two things are infinite: the universe and hum...,Albert Einstein,"['human-nature', 'humor', 'infinity', 'philoso..."
3,"“so many books, so little time.”",Frank Zappa,"['books', 'humor']"
4,“a room without books is like a body without a...,Marcus Tullius Cicero,"['books', 'simile', 'soul']"
...,...,...,...
2503,“morality is simply the attitude we adopt towa...,"Oscar Wilde,","['morality', 'philosophy']"
2504,“don't aim at success. the more you aim at it ...,"Viktor E. Frankl,","['happiness', 'success']"
2505,"“in life, finding a voice is speaking and livi...",John Grisham,['inspirational-life']
2506,"“winter is the time for comfort, for good food...",Edith Sitwell,"['comfort', 'home', 'winter']"


In [ ]:
from datasets import load_dataset,Dataset
# data_path="/kaggle/input/english-quotes/quotes.jsonl"
# data = load_dataset("json",data_files={
#     "train":data_path
# })
# data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
data = data[['quote','tags']]
data1 = Dataset.from_pandas(data, split='train')
data1

Dataset({
    features: ['quote', 'tags'],
    num_rows: 2508
})

## step 2: Loading the Llama-2 LLM for downstreaming task

In [ ]:
!pip install -q -U bitsandbytes
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
  # !pip uninstall pyarrow
!pip install pyarrow==12.0.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# EleutherAI/gpt-neo-125m
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

## step 3: converting normal model to PEFT model using LORA config

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(r=8,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"], #specific to Llama models.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM")

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3508801536 || trainable%: 0.23907331075678143


In [ ]:
!pip install datasets

In [ ]:
data1 = data1.map(lambda samples: tokenizer(samples["quote"]), batched=True)
data1

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

## step 4: training the model

In [ ]:
import transformers

# needed for Llama tokenizer
tokenizer.pad_token = tokenizer.eos_token # </s>

trainer = transformers.Trainer(
    model=model,
    train_dataset=data1,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.291100
2,2.352900
3,2.539200
4,2.556700
5,1.982500
6,1.601100
7,2.022400
8,2.050200
9,1.764600
10,1.756600


TrainOutput(global_step=10, training_loss=2.091718888282776, metrics={'train_runtime': 61.574, 'train_samples_per_second': 0.65, 'train_steps_per_second': 0.162, 'total_flos': 36398413479936.0, 'train_loss': 2.091718888282776, 'epoch': 0.02})

In [ ]:
model.config.use_cache = True
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer)

In [ ]:
# output_dir='/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation_adaptive_model'
# model.save_pretrained(adapter_model)

##step 6 : converting normal model to PEFT model

In [ ]:
from transformers import AutoModelForCausalLM
model_id = "/content/drive/MyDrive/Colab Notebooks/NLP/LLM/Llama-2-7b-chat-hf-sharded-bf16"
model_1 = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16, cache_dir="cache")

from peft import PeftModel

# load perf model with new adapters
model_peft = PeftModel.from_pretrained(model_1,model)
model = model.merge_and_unload()
model_peft.save_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation')

##step 7 : loading the final saved model


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

finetuned_model_checkpoint = '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation'

bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(finetuned_model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(finetuned_model_checkpoint, quantization_config=bitsandbytes_config)

## creating the prompts and streaming the generated text​

In [ ]:
from transformers import TextStreamer
def stream(user_prompt,model):
    runtimeFlag = "cuda:0"
    system_prompt = 'You are a helpful assistant that provides accurate and concise responses'

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer)
    print('the quote is generating')
    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=100)

## English Quote Genereted

In [ ]:
stream('Provide a quote on love and wind it should contains less words but interesting',model)

the quote is generating
<s> [INST] <<SYS>>
You are a helpful assistant that provides accurate and concise responses
<</SYS>>

Provide a quote on love and wind it should contains less words but interesting [/INST]

"Love is like a gentle breeze, it can calm the soul and lift the spirit. Just as wind can carry the scent of blooming flowers, love carries the essence of connection and belonging."</s>


In [1]:
!pip install -q -U bitsandbytes
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
model_name = 'Trelis/Llama-2-7b-chat-hf-sharded-bf16'
peft_model = '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation2'

In [8]:
from transformers import AutoModelForCausalLM,AutoTokenizer
from peft import PeftModel
import torch

In [9]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"":0},
)
model = PeftModel.from_pretrained(base_model, peft_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3')

('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3/tokenizer.json')

In [13]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/LLM/quote_generation3')

In [ ]:
!pip install streamlit langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

## function to get the response

## creating the UI

In [ ]:
st.set_page_config(page_title='Generate Quote',page_icon=':)',layout='centered',initial_sidebar_state='collapsed')
st.heads("Generated Quote")
input_txet = st.text_input("Enter the genre")

AttributeError: module 'streamlit' has no attribute 'heads'

In [ ]:
col1,col2 = st.columns([5,5])
with col1:
  no_words = st.text_input("no of word")
with col2:
  quote_style = st.selectbox('writing the quote for ',('new','old'),index=0)

submi = st.button('generate')


In [ ]:
if submit:
